In [81]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import tweepy

import coinbasepro as cbp

import requests
import datetime
from datetime import datetime, timezone, date, timedelta
import time

from textblob import TextBlob
import re
import string

from numpy import mean
from numpy import median

import csv

import json

from pytrends.request import TrendReq

client = cbp.PublicClient()


In [88]:
df = pd.read_csv('data/bitcoin.csv')

In [89]:
df.head()

,time,low,high,open,close,volume,greed_and_fear,google_trend,sentiment_medio,difficulty
0,2022-12-11,17073.19,17271.92,17128.10,17085.21,9948.852670,26,30,0.057855,3.424433e+13
1,2022-12-10,17093.42,17227.64,17130.49,17128.10,7860.586876,27,28,0.023664,3.424433e+13
2,2022-12-09,17060.69,17352.62,17226.03,17130.59,20976.636999,26,28,0.079547,3.424433e+13
3,2022-12-08,16738.00,17300.59,16839.76,17226.01,23533.234537,25,28,0.098894,3.424433e+13
4,2022-12-07,16679.52,17140.22,17089.18,16840.00,22635.468488,29,28,0.098525,3.424433e+13


In [4]:
X = df[['low', 'high', 'open', 'volume', 'greed_and_fear', 'google_trend', 'sentiment_medio', 'difficulty']]

In [5]:
y = df['close']


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model = RandomForestRegressor()

In [8]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [9]:
model.score(X_test, y_test)

0.9186523862664571

In [15]:
domani = pd.DataFrame([[df['low'][10], df['high'][10], df['open'][10], df['volume'][10], df['greed_and_fear'][10], df['google_trend'][10], df['sentiment_medio'][10], df['difficulty'][10]]], columns=['low', 'high', 'open', 'volume', 'greed_and_fear', 'google_trend', 'sentiment_medio', 'difficulty'])
previsione = model.predict(domani)[0]

In [16]:
print(previsione)

17026.421300000016


**Scarico prezzi bitcoin di oggi**

In [54]:
# Scarica i prezzi di bitcoin per l'ultimo anno
candles = client.get_product_historic_rates(product_id="BTC-USD", start="2022-12-21T00:00:00", stop="2022-12-22T00:00:00", granularity=86400)

In [55]:
df_domani = pd.DataFrame(candles)
print(df_domani.shape)
df_domani.head()

(2, 6)


,time,low,high,open,close,volume
0,2022-12-22,16560.84,16868.82,16825.18,16815.51,21269.7673979
1,2022-12-21,16725.78,16926.05,16896.17,16826.56,16743.62189364


**Scarico greed_and_fear di oggi**

In [56]:
# Inizializza la lista data_list
data_list = []

# Effettua 31 chiamate all'API di Alternative.me
for i in range(2):
  # Prepara la richiesta HTTP
  url = "https://api.alternative.me/fng/?limit=204&timestamp=" + str(i)
  headers = {'Content-Type': 'application/json'}

  # Effettua la richiesta HTTP
  response = requests.get(url, headers=headers)

  # Carica i dati in formato JSON
  data = response.json()

  # Estrai il valore "value" e il timestamp
  value = data["data"][i]["value"]
  timestamp = data["data"][i]["timestamp"]

  # Aggiungi i valori "value" e "timestamp" nella lista data_list
  data_list.append((value, timestamp))

  # Per ogni tupla nella lista data_list
for value, timestamp in data_list:
  # Converti il timestamp in una data
  date = datetime.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d')
  #date = pd.to_datetime(date)
  # Seleziona la riga del dataframe corrispondente alla data
  df_domani.loc[df_domani['time'] == date, 'greed_and_fear'] = value

# Stampa il dataframe aggiornato
df_domani.head(20)

,time,low,high,open,close,volume,greed_and_fear
0,2022-12-22,16560.84,16868.82,16825.18,16815.51,21269.7673979,28
1,2022-12-21,16725.78,16926.05,16896.17,16826.56,16743.62189364,NaN


**Scarico difficulty di oggi**

In [58]:
# Carica il file JSON in un oggetto python
with open('data/difficulty.json', 'r') as file:
    data = json.load(file)

# Normalizza i dati del file JSON
df_difficulty = pd.json_normalize(data, 'difficulty', ['metric1', 'metric2'])

# Crea una lista di date partendo dal 23 dicembre 2021 fino al 22 dicembre 2022
dates = pd.date_range(start='2021-12-24', end='2022-12-22', freq='D')

# Crea una nuova colonna "date" nel tuo dataframe e assegna ad ogni riga una data
df_difficulty['date'] = dates

df_difficulty = df_difficulty.drop(['x', 'metric1', 'metric2'], axis=1)

df_difficulty.rename(columns={'date': 'time'}, inplace=True)

#trasforma df['time'] in datetime
df_difficulty['time'] = pd.to_datetime(df_difficulty['time'])

#seleziono solo la data di domani
#df_difficulty = df_difficulty.loc['2022-12-21':'2022-12-12']

df_difficulty = df_difficulty.iloc[::-1]

df_domani = pd.merge(df_domani, df_difficulty, on='time')

df_domani.head()

,time,low,high,open,close,volume,greed_and_fear,y
0,2022-12-22,16560.84,16868.82,16825.18,16815.51,21269.7673979,28,3.536407e+13
1,2022-12-21,16725.78,16926.05,16896.17,16826.56,16743.62189364,NaN,3.536407e+13


In [59]:
df_domani.rename(columns={'y': 'difficulty'}, inplace=True)
df_domani.head()

,time,low,high,open,close,volume,greed_and_fear,difficulty
0,2022-12-22,16560.84,16868.82,16825.18,16815.51,21269.7673979,28,3.536407e+13
1,2022-12-21,16725.78,16926.05,16896.17,16826.56,16743.62189364,NaN,3.536407e+13


**scarico sentiment di oggi**

In [77]:
#istanzio l'auth
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPgukgEAAAAADYQXq5tFdfahfgxE%2FSQqxcV4OFU%3DIETAxHE05aQI4saJCxFJMQiaurCSZgYMSHZ9kl7SMVNNxM5V4S"  # BEARER_TOKEN
auth = tweepy.Client(bearer_token)
api = tweepy.API(auth)

#scarico tweets di oggi

# Numero di giorni da scaricare
num_days = 1

# Numero di tweet da scaricare ogni giorno
num_tweets = 500

# Inizializza una lista che conterrà i tweet scaricati
tweet_list = []

# Inizializza la data di inizio del loop
start_date = today = datetime.today() - timedelta(days=2)
#start_date = datetime.now() - timedelta(days=30)

# Crea il loop
for i in range(num_days):
    # Aggiorna la data di fine
    end_date = start_date + timedelta(days=1)

    # Formatta le date come stringhe nel formato richiesto dall'API
    formatted_start_date = start_date.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%dT%H:%M:%S")

    #hashtag
    query= "bitcoin"

    # Scarica i tweet
    response = auth.search_all_tweets(query, max_results=num_tweets, tweet_fields=["created_at", "lang"], start_time=start_date, end_time=end_date)
    tweets = response.data

    # Aggiungi i tweet alla lista
    for tweet in tweets:
        tweet_list.append([tweet.created_at, tweet.id, tweet.text, tweet.lang])

    # Aggiorna la data di inizio
    start_date = end_date
    time.sleep(5)

df_sentiment = pd.DataFrame(data=tweet_list, columns=['created_at', 'tweet_id', 'text', 'language'])  
df_sentiment.set_index("created_at", inplace = True)

df_sentiment = df_sentiment.iloc[::-1]

def preprocess_tweet(tweet):
    # Rimuove i caratteri di nuova riga e a capo
    tweet = tweet.replace("\n", " ").replace("\r", " ")
    
    # Rimuove i link
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Rimuove gli hashtag
    tweet = re.sub(r"#\S+", "", tweet)
    
    # Rimuove le menzioni
    tweet = re.sub(r"@\S+", "", tweet)
    
    # Rimuove i caratteri di punteggiatura
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    
    # Converti il testo in minuscolo
    tweet = tweet.lower()
    
    return tweet

# Crea una copia della colonna 'text' del dataframe
df_sentiment['text_clean'] = df_sentiment['text'].copy()

# Sostituisci i testi elaborati nella colonna 'text_clean' del dataframe
df_sentiment['text_clean'] = df_sentiment['text_clean'].apply(preprocess_tweet)

def get_sentiment(tweet_text):
    analysis = TextBlob(tweet_text)
    return analysis.sentiment.polarity

df_sentiment['sentiment'] = df_sentiment['text_clean'].apply(get_sentiment)

df_sentiment = df_sentiment.reset_index()
df_sentiment['created_at'] = pd.to_datetime(df_sentiment['created_at'])

# Crea una nuova colonna 'date' che estrae la data dalla colonna 'created_at'
df_sentiment['date'] = df_sentiment['created_at'].dt.date

# raggruppa i dati per data utilizzando la colonna 'data' come chiave di raggruppamento
grouped_data = df_sentiment.groupby('date')

# per ogni gruppo di dati, calcola la mediana del sentiment e aggiungi il valore alla lista medie
medie = []
for name, group in grouped_data:
    medie.append(mean(group['sentiment']))

# crea il dizionario con le date come chiavi e i valori medi come valori
dati = {key: value for key, value in zip(grouped_data.groups.keys(), medie)}

# crea il DataFrame utilizzando il dizionario appena creato
df_medie = pd.DataFrame.from_dict(dati, orient='index')

# rinomina la colonna del DataFrame con il nome desiderato
df_medie.rename(columns={0: 'sentiment_medio'}, inplace=True)

df_medie = df_medie.reset_index()

df_medie.rename(columns={'index': 'time'}, inplace=True)

df_medie = df_medie.iloc[::-1]

#trasformo la colonna created_at in datetime
df_medie['time'] = pd.to_datetime(df_medie['time'])

#df_medie = df_medie.loc['2022-12-21':'2022-12-23']

df_medie


,time,sentiment_medio
0,2022-12-22,0.052413


In [78]:
#trasforma df['time'] in datetime
df_domani['time'] = pd.to_datetime(df_domani['time'])

# Unisci i due dataframe utilizzando la colonna "time" come chiave di unione
df_domani = pd.merge(df_domani, df_medie, on='time')

In [79]:
df_domani.head()

,time,low,high,open,close,volume,greed_and_fear,difficulty,sentiment_medio
0,2022-12-22,16560.84,16868.82,16825.18,16815.51,21269.7673979,28,3.536407e+13,0.052413


**scarico il google trend di oggi**

In [86]:
pytrends = TrendReq(hl='en-US', tz=360)

# build payload

kw_list = ["bitcoin"] # list of keywords to get data 

pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m')

#1 Interest over Time
data = pytrends.interest_over_time()

#riempio le date vuote con l'ultimo dato registrato
df_google = data.resample('D').ffill()

df_google = df_google.reset_index()

df_google.tail()

,date,bitcoin,isPartial
353,2022-12-14,30,False
354,2022-12-15,30,False
355,2022-12-16,30,False
356,2022-12-17,30,False
357,2022-12-18,27,True


**Prevedo il prezzo di chiusura di oggi**

In [90]:
domani = pd.DataFrame([[df_domani['low'][0], df_domani['high'][0], df_domani['open'][0], df_domani['volume'][0], df_domani['greed_and_fear'][0], df['google_trend'][0], df_domani['sentiment_medio'][0], df_domani['difficulty'][0]]], columns=['low', 'high', 'open', 'volume', 'greed_and_fear', 'google_trend', 'sentiment_medio', 'difficulty'])
previsione = model.predict(domani)[0]
print(previsione)

16635.147100000006
